In [1]:
import org.apache.spark.sql.types.{StructType, StringType}
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.sql.functions._
import org.apache.spark.sql.DataFrame

In [2]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.4


In [3]:
val inputStreamPath = "/home/jovyan/data/events-stream"
val modelPath = "/home/jovyan/models/spark-ml-model"

val dataSchema = new StructType()
    .add("tweet", StringType)

val inputDF = spark
    .readStream
    .schema(dataSchema)
    .option("maxFilesPerTrigger", 1)
    .json(inputStreamPath)

inputStreamPath = /home/jovyan/data/events-stream
modelPath = /home/jovyan/models/spark-ml-model
dataSchema = StructType(StructField(tweet,StringType,true))
inputDF = [tweet: string]


[tweet: string]

In [4]:
import org.apache.spark.sql.functions._

val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))
val model = PipelineModel.load(modelPath)

val predictionsDF = model
    .transform(inputDF)
    .select($"tweet",
            $"prediction",
            getProbability($"probability").alias("clean_probability"))

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))
model = pipeline_fafabf1551e4
predictionsDF = [tweet: string, prediction: double ... 1 more field]


[tweet: string, prediction: double ... 1 more field]

In [5]:
predictionsDF
    .writeStream
    .outputMode("append")
    .format("console")
    .start()

org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@2824a763

-------------------------------------------
Batch: 0
-------------------------------------------+--------------------+----------+-------------------+
|               tweet|prediction|  clean_probability|
+--------------------+----------+-------------------+
|@TheCavalry I'm a...|       0.0|0.48957323034707867|
|painful, emotiona...|       0.0| 0.4690013796734148|
|Saddened by the A...|       1.0| 0.5110175201989817|
|I really really w...|       0.0| 0.4237319079443053|
|Misses his dance ...|       1.0| 0.5110175201989817|
|doesn't know how ...|       1.0| 0.5110175201989817|
|HAPPY FATHERS DAY...|       1.0| 0.5316863485936643|
|hmmm...coffee bea...|       1.0| 0.5298811809059409|
|at the pool...fee...|       0.0|0.48064878380281967|
|Sunny canals in A...|       1.0| 0.5110175201989817|
|@karaoconnell And...|       1.0| 0.5014850350429527|
|whers the sun gon...|       1.0|  0.514524088806193|
|@patricknorton my...|       0.0| 0.4287369027863586|
|loved the day of ...|       1.0| 0.5240